In [1]:
import sys
sys.path.append("..")
import ScraperFC as sfc # import local ScraperFC
from ScraperFC.shared_functions import sources, InvalidYearException, UnavailableSeasonException

import datetime
import itertools
from tqdm import tqdm
import traceback

In [3]:
link = 'https://fbref.com/en/matches/8b005381/Rangers-NK-Maribor-August-1-2001-Champions-League'
scraper = sfc.FBRef()
try:
    match = scraper.scrape_match(link)
    # links = scraper.get_match_links(2014, 'EFL Championship')
except:
    traceback.print_exc()
finally:
    scraper.close()



====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\oseymour\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache


In [7]:
type(match['Shots'].values[0])

pandas.core.series.Series

In [ ]:
leagues = sources["FBRef"].keys()
years = range(1988, datetime.now().year+1)
iterator = list(itertools.product(leagues, years))


scraper = sfc.FBRef()
try:
    for (league, year) in tqdm(iterator):
        year = int(year) # year became a string during random sampling?
        league = str(league) # league also became a weird type of numpy string?
        print(year, league)

        # Skip invalid years
        try:
            sfc.shared_functions.check_season(year, league, "FBRef")
        except InvalidYearException:
            continue

        try:
            season_link = scraper.get_season_link(year, league)
        except UnavailableSeasonException:
            continue

        match_links = scraper.get_match_links(year, league)

        assert type(season_link) is str

        assert len(match_links) > 0
finally:
    scraper.close()